In [1]:
import numpy as np
from scene import Scene
import tensorflow as tf
from tf_agents.networks.q_network import QNetwork
from tf_agents.specs import tensor_spec
from snake_game import SnakeGame

scene = Scene(using_cnn=True, init_randomly=True)
episodes_count = 100000

pygame 2.1.3 (SDL 2.0.22, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from tf_agents.environments.tf_py_environment import TFPyEnvironment

env = SnakeGame(scene)
env = TFPyEnvironment(env)

In [3]:
conv_layer_params = [(32, (2, 2), 1), (32, (2, 2), 1)]
fc_layer_params = [64]

q_net = QNetwork(
    env.observation_spec(),
    env.action_spec(),
    conv_layer_params=conv_layer_params,
    fc_layer_params=fc_layer_params)

In [4]:
# Create the agent
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tensorflow import keras
from keras.optimizers.legacy import Adam
from keras.losses import Huber
from tf_agents.trajectories import TimeStep

train_step_counter = tf.Variable(0)
optimizer = Adam(learning_rate=0.001)

epsilon = lambda train_step: (1 / (tf.cast(tf.linspace(1, 8, episodes_count)**3, tf.float32)))[train_step]
# epsilon = keras.optimizers.schedules.PolynomialDecay(
#     initial_learning_rate=1.0,
#     decay_steps=episodes_count,
#     end_learning_rate=0.01
# )

agent = DqnAgent(
    time_step_spec=env.time_step_spec(),
    action_spec=env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=Huber(reduction="none"),
    gamma=tf.constant(0.95, dtype=tf.float32),
    train_step_counter=train_step_counter,
    epsilon_greedy=lambda: epsilon(train_step_counter)
)

In [5]:
# Create the replay buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=100000
)

In [6]:
# Create the training metrics
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric()
]

In [7]:
# Create the driver
from tf_agents.drivers import dynamic_step_driver

collect_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=agent.collect_policy,
    observers=[replay_buffer.add_batch] + train_metrics,
    num_steps=1
)

# Run a random policy to fill the replay buffer
from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(env.time_step_spec(), env.action_spec())
init_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=initial_collect_policy,
    observers=[replay_buffer.add_batch] + train_metrics,
    num_steps=1
)

final_time_step, final_policy_state = init_driver.run()

In [8]:
# Create the dataset
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2, # Capaz cambiar esto
    num_parallel_calls=3
).prefetch(3)

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [9]:
# Create the training loop

from tf_agents.utils.common import function
from tf_agents.trajectories import trajectory
from tf_agents.eval.metric_utils import log_metrics
import logging

logging.getLogger().setLevel(logging.INFO)

collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

def train_agent(n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(env.batch_size)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        if iteration % 1000 == 0:
            print("Iteration: ", iteration)
            log_metrics(train_metrics)

In [10]:
train_agent(episodes_count)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


INFO:absl: 
		 NumberOfEpisodes = 1
		 EnvironmentSteps = 2
		 AverageReturn = -30.200000762939453
		 AverageEpisodeLength = 1.0


Iteration:  0


INFO:absl: 
		 NumberOfEpisodes = 184
		 EnvironmentSteps = 1002
		 AverageReturn = -4686.7568359375
		 AverageEpisodeLength = 6.199999809265137


Iteration:  1000


...

Iteration:  97000


INFO:absl: 
		 NumberOfEpisodes = 3683
		 EnvironmentSteps = 98002
		 AverageReturn = 185322.71875
		 AverageEpisodeLength = 44.29999923706055


Iteration:  98000


INFO:absl: 
		 NumberOfEpisodes = 3708
		 EnvironmentSteps = 99002
		 AverageReturn = 188117.53125
		 AverageEpisodeLength = 45.20000076293945


Iteration:  99000


In [11]:
# Evaluate the agent

time_step = env.reset()
rewards = []
steps = 0

while not time_step.is_last() and steps < 200:
    steps += 1
    print("Map:\n", np.argmax(time_step.observation.numpy(), axis=3))
    action_step = agent.policy.action(time_step)
    print("Action: ", action_step.action.numpy())
    time_step = env.step(action_step.action)
    rewards.append(time_step.reward.numpy()[0])
    
print("Total reward: ", sum(rewards))
print("Total steps: ", steps)

Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 3 0 1 2]
  [0 0 0 0 0]]]
Action:  [0]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 3 1 2 0]
  [0 0 0 0 0]]]
Action:  [0]
Map:
 [[[3 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 1 2 2 0]
  [0 0 0 0 0]]]
Action:  [0]
Map:
 [[[3 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [1 2 2 0 0]
  [0 0 0 0 0]]]
Action:  [1]
Map:
 [[[3 0 0 0 0]
  [0 0 0 0 0]
  [1 0 0 0 0]
  [2 2 0 0 0]
  [0 0 0 0 0]]]
Action:  [1]
Map:
 [[[3 0 0 0 0]
  [1 0 0 0 0]
  [2 0 0 0 0]
  [2 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [1]
Map:
 [[[1 0 0 3 0]
  [2 0 0 0 0]
  [2 0 0 0 0]
  [2 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[2 1 0 3 0]
  [2 0 0 0 0]
  [2 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[2 2 1 3 0]
  [2 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[2 2 2 1 0]
  [2 0 0 0 0]
  [0 0 0 3 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [3]
Map:
 [[[2 2 2 2 0]
  [0 0 0 1 0]
  [0 0 0 3 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [3]

In [12]:
# Count the average steps per episode

n = 500
total_steps = 0
steps = 0
total_500 = 0

for _ in range(n):
    time_step = env.reset()
    steps = 0
    while not time_step.is_last() and steps < 500:
        steps += 1
        action_step = agent.policy.action(time_step)
        time_step = env.step(action_step.action)
    
    if steps == 500:
        total_500 += 1
    else:
        total_steps += steps
        
if steps == 500:
    print("500 episodes reached")
    
print("Average steps per episode: ", total_steps / (n - total_500))
print("Total 500 episodes: ", total_500)

Average steps per episode:  51.354
Total 500 episodes:  0


In [13]:
print("Map:\n", np.argmax(time_step.observation.numpy(), axis=3))

Map:
 [[[0 0 0 0 0]
  [2 0 3 0 0]
  [1 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]


In [14]:
action_step = agent.policy.action(time_step)
time_step = env.step(action_step.action)
print("Map:\n", np.argmax(time_step.observation.numpy(), axis=3))

Map:
 [[[0 0 0 0 0]
  [1 0 3 0 0]
  [2 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]


In [15]:
# Save the agent
import os
from tf_agents.policies.policy_saver import PolicySaver

saver = PolicySaver(agent.policy, batch_size=None)
saver.save("policy")

INFO:absl:Function `function_with_signature` contains input name(s) 0/step_type, 0/reward, 0/discount, 0/observation, 3755814, 3755816, 3755818, 3755820, 3755822, 3755824, 3755826, 3755828 with unsupported characters which will be renamed to step_type, reward, discount, observation, unknown, unknown_0, unknown_1, unknown_2, unknown_3, unknown_4, unknown_5, unknown_6 in the SavedModel.
INFO:absl:Function `function_with_signature` contains input name(s) 3755869 with unsupported characters which will be renamed to unknown in the SavedModel.


INFO:tensorflow:Assets written to: policy/assets


/Users/juanpablo/miniconda3/lib/python3.11/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:475: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
INFO:tensorflow:Assets written to: policy/assets
